# MPCS 51042-2 Lecture 4
# Higher-Order Functions
## Ron Rahaman - University of Chicago, Department of Computer Science
## Oct 28, 2019

# Functions as Objects

## A function is a first-class object

* This `def` statement creates a variable named `factorial` and new instance of a function object. 
* The name and the function object do not exist until the `def` statement is executed.

In [1]:
def factorial(n):
    res = 1
    for i in range(2, n+1):
        res *= i
    return res

The [`type` built-in function](https://docs.python.org/3/library/functions.html#type) shows that the name `factorial` refers to an instance of a function object.

In [4]:
type(factorial)

function

## The call operator

The call operator `()` executes a function object (including passing arguments and returning a result).

In [5]:
type(factorial(4))

int

Any class for which the call operator is implemented is **callable**.  This includes:
* User defined functions (`def` and `lambda`)
* User defined classes and class methods
* Built-in functions, classes, and class methods (`len`, `zip`, and `list.append`)
* Generators

In most cases, the exact type of a callable won't matter to the user.

## Aside: Attributes of Functions

The [built-in `dir` function](https://docs.python.org/3/library/functions.html#dir) returns the attributes of any object.  For a function, it will show many attributes specific to the callables.  

In [2]:
dir(factorial)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

## Aside: Some Interesting Function Attributes

* The `__code__` attribute has metadata about the function.  See Ramalho Ch. 5 and the [`inspect` module](https://docs.python.org/3/library/inspect.html) for more info

In [3]:
factorial.__code__.co_varnames   # Local variable names

('n', 'res', 'i')

In [4]:
factorial.__code__.co_argcount  # Number of args

1

# Higher-Order Functions

## What are Higher-Order Functions?

* A **higher-order function** does either or both of the following:
  * Takes an instance of a function object as an argument
  * Returns an instance of a fuction object as a result.
* Some built-in examples
  * `map`, `filter`, and `reduce`: take a function object as the first parameter
  * `sorted`: takes a function object as the `key` parameter

## A User-Defined Higher-Order Function

Every call to `make_averager` does the following:
  * Creates a new list named `series` that is local to `make_averager`.  
  * Creates a new function object named `averager` that is local to `make_averager`
  * Returns the new function object
  
Note that the body of `make_averager` **does not call `averager`**

This is sometimes called a **factory function**.  

In [7]:
def make_averager(): 
    series = []
    def averager(new_value): 
        series.append(new_value) 
        total = sum(series) 
        return total/len(series)
    return averager

## Closures

A **closure** consists of an instance of a function object and its external name references.  
* This includes nonlocal, global, and built-in functions

Here, the closure of the function object named `averager` includes:
* A reference to a nonlocal list (`series`)
* References to built-in functions and methods (`sum`, `len`, `append`).

In [8]:
def make_averager(): 
    series = []
    def averager(new_value): 
        series.append(new_value) 
        total = sum(series) 
        return total/len(series)
    return averager

## Using `make_averager`

This statement calls `make_averager` and assigns the resulting function object to the variable `my_avg_1`.

In [9]:
my_avg_1 = make_averager()

Since `my_avg_1` refers to a function object, it is callable. 

In [10]:
my_avg_1(10)  # This returns the running average each time we call

In [11]:
my_avg_1(11)

10.5

In [12]:
my_avg_1(12)

11.0

## Multiple Instances of Function Objects

Every call to `my_averager` returns a new instance of a function object, each with a separate closure.

In [13]:
my_avg_1 = make_averager()
my_avg_2 = make_averager()

In [14]:
my_avg_1(10)
my_avg_1(11)

10.5

In [15]:
my_avg_2(4)
my_avg_2(5)

4.5

In [16]:
my_avg_1(12)

11.0

## Aside: Inspecting the Closure

* A function's [`__closure__` attribute](https://docs.python.org/3/reference/datamodel.html?highlight=__closure__) refers to the variables in the closures.  
* The [`inspect.getclosurevars()` function](https://docs.python.org/3/library/inspect.html#inspect.getclosurevars) is an easier way to see the closure. 
 * Includes nonlocals, globals, builtins, and unbound methods (see OO lectures).

In [17]:
my_avg = make_averager()
my_avg(10), my_avg(11), my_avg(12)

(10.0, 10.5, 11.0)

In [18]:
from inspect import getclosurevars
for v in getclosurevars(my_avg):
    print(v)

{'series': [10, 11, 12]}
{}
{'sum': <built-in function sum>, 'len': <built-in function len>}
{'append'}


# Higher-Order Functions That Take Functions as Args

## Functions as Args

In this version:
* Both `series` and `reduction_func` are local to `make_reducer`
* Both `series` and `reduction_func` are nonlocal to `reducer`
* Both `series` and `reduction_func` are in the closure of the returned function object

In [19]:
def make_reducer(reduction_func): 
    series = []
    def reducer(new_value): 
        series.append(new_value) 
        return reduction_func(series) 
    return reducer

## Using `make_reducer`

* Each time we call `make_reducer`, the following happens:
    * We can pass a different reduction function
    * Then each instance of the returned function will have a different reduction function in its closure

In [20]:
sum_reducer = make_reducer(sum)
sum_reducer(4), sum_reducer(5), sum_reducer(6)

(4, 9, 15)

In [21]:
import numpy as np
med_reducer = make_reducer(np.median)
med_reducer(4), med_reducer(5), med_reducer(3)

(4.0, 4.5, 4.0)

Do we need to keep all previous values in `series`?

## Aside: The Closure of Result of `make_reducer`

In [22]:
def make_reducer(reduction_func): 
    series = []
    def reducer(new_value): 
        series.append(new_value) 
        return reduction_func(series) 
    return reducer

In [23]:
import numpy as np
med_reducer = make_reducer(np.median)
med_reducer(4), med_reducer(5), med_reducer(3)

(4.0, 4.5, 4.0)

The closure of `med_reducer` includes the nonlocal variables `reduction_func` and `series`

In [24]:
from inspect import getclosurevars
for k, v in getclosurevars(med_reducer).nonlocals.items():
    print(k, v, sep=' : ')

reduction_func : <function median at 0x11186df28>
series : [4, 5, 3]


## Generalized Argument Passing for Factory Functions

## Review:  `*args` and `**kwargs` in function definitions.  

See Lutz, Table 18-1

| Syntax | Action |
|--------|--------|
| `def func(arg)` | Matches arguments by position or keyword |
| `def func(arg=value)` | Default value if `arg` is not passed by caller |
| `def func(*args)` | Packs remaining positional arguments in a tuple named `args` |
| `def func(**kwargs)` | Packs remaining keyword arguments in a dict named `kwargs`|

Must be listed in this order:
1. All normal arguments (`arg`)
2. All default arguments (`arg=value`)
3. All arguments packed by position (`*args`)
4. All arguments packed by keyword (`**kwargs`)

## Using `*args` and `**kwargs` in function definitions

In [5]:
def print_args(*args, **kwargs):
    print(args)   # args is a tuple
    print(kwargs) # kwargs is a dict

When collecting positional and keyword arguments, you can pass arbitrary combinations.

In [6]:
print_args(1, 2, 3, a='apple', b='banana')

(1, 2, 3)
{'a': 'apple', 'b': 'banana'}


In [27]:
print_args(1, 2)

(1, 2)
{}


In [28]:
print_args(a='apple', b='banana', c='cherry')

()
{'a': 'apple', 'b': 'banana', 'c': 'cherry'}


## Review:  `*args` and `**kwargs` in function calls.  

See Lutz, Table 18-1

| Syntax | Action |
|--------|--------|
| `func(arg)` | Argument is matched by position |
| `func(arg=value)` | Argument is matched by keyword |
| `func(*args)` | Unpacks an iterable into individual positional args |
| `func(**kwargs)` | Unpacks a dict into individual keyword args |

Must appear in this order:
* All positional arguments (`arg`)
* Any combination of keyword arguments (`arg=value`) and upacked iterables (`*args`)
* All unpacked dicts (`**kwargs`)

## Example:  Unpacking in `print()`

Since `print` takes an arbitrary number of positional arguments, we can unpack iterables when we call it

In [29]:
L = ['foo', 'bar', 'baz']
print(*L, sep=' and ')

foo and bar and baz


We can also combine it with dict unpacking

In [30]:
delims = {'sep':' and ', 'end':'!'}
print(*L, **delims)

foo and bar and baz!

## Arbitrary Args in Factory Function

In [8]:
from datetime import datetime
def make_timelogger(func):
    times = []
    def timelogger(*args, **kwargs):
        times.append(datetime.now())
        return func(*args, **kwargs)
    return timelogger

1. The factory function takes an arbitrary function (`func`) and returns a funcion instance.  
2. Both `func` and `times` are in the closure of the returned function object.
3. When you call the returned function object, it packs arbitrary arguments into `args` and `kwargs`.
3. When `func` is called, `args` and `kwargs` are unpacked

## The Closure of the `print_logger`

In [9]:
print_logger = make_timelogger(print)

print_logger("Hello!")
foods = input("What did you eat? ").split()
print_logger("Here's what you ate: ", end="")
print_logger(*foods, sep=', ')

Hello!
What did you eat? wqeq
Here's what you ate: wqeq


In [33]:
for k, v in getclosurevars(print_logger).nonlocals.items():
    print(k, v, sep=' : ')

func : <built-in function print>
times : [datetime.datetime(2019, 10, 28, 16, 25, 57, 909938), datetime.datetime(2019, 10, 28, 16, 26, 4, 306518), datetime.datetime(2019, 10, 28, 16, 26, 4, 306661)]


## Making Your Own Function Attributes

* For the user, it is not ideal to inspect the closure via the `__closure__` attribute or `getclosurevars` function
* You can define your own attributes simply by assignment

In [10]:
from datetime import datetime
def make_timelogger(func):
    times = []
    def timelogger(*args, **kwargs):
        times.append(datetime.now())
        return func(*args, **kwargs)
    timelogger.times = times
    return timelogger

## Using Our Function Attribute

In [12]:
print_logger = make_timelogger(print)

print_logger("Hello!")
foods = input("What did you eat? ").split()
print_logger("Here's what you ate: ", end="")
print_logger(*foods, sep=', ')

Hello!
What did you eat? 2e1e 21e 11 
Here's what you ate: 2e1e, 21e, 11


In [36]:
print_logger.times

[datetime.datetime(2019, 10, 28, 16, 26, 4, 340640),
 datetime.datetime(2019, 10, 28, 16, 26, 8, 502285),
 datetime.datetime(2019, 10, 28, 16, 26, 8, 502769)]

How would you display or return `times` and make it read-only?